<a href="https://colab.research.google.com/github/AbrahamtheAraiza99/inteligencia-artificial/blob/master/8PUZZLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from enum import Enum
from queue import Queue
import heapq
import time

# Tipos de movimientos posibles
class MovementType(Enum):
    UP = "UP"
    DOWN = "DOWN"
    LEFT = "LEFT"
    RIGHT = "RIGHT"

# Heurísticas utilizadas en algunos algoritmos
class Heuristic(Enum):
    H_ONE = 1
    H_TWO = 2
    H_THREE = 3

# Nodo que representa el estado, su costo, y el nodo anterior
class Node:
    def __init__(self, state, parent=None, cost=0, depth=0):
        self.state = state
        self.parent = parent
        self.children = []
        self.cost = cost
        self.estimated_cost_to_goal = 0
        self.total_cost = 0
        self.depth = depth
        self.visited = False

    def set_total_cost(self, path_cost, est_cost=0):
        self.total_cost = path_cost + est_cost

    def path(self):
        node, p = self, []
        while node:
            p.append(node)
            node = node.parent
        return p[::-1]

    def __lt__(self, other):
        return self.total_cost < other.total_cost

# Función para obtener los sucesores (nuevos estados)
def get_successors(state):
    successors = []
    index = state.index('0')  # El espacio vacío está representado por '0'

    # Definir posibles movimientos
    swap = lambda s, i, j: ''.join([s[j] if k == i else s[i] if k == j else s[k] for k in range(len(s))])

    # Mapa de posiciones válidas
    positions = {
        0: [1, 3],
        1: [0, 2, 4],
        2: [1, 5],
        3: [0, 4, 6],
        4: [1, 3, 5, 7],
        5: [2, 4, 8],
        6: [3, 7],
        7: [4, 6, 8],
        8: [5, 7]
    }

    for target in positions[index]:
        successors.append(swap(state, index, target))
    return successors

# Función para identificar el tipo de movimiento (arriba, abajo, izquierda, derecha)
def find_transition(source, destination):
    diff = destination.index('0') - source.index('0')
    return {
        -3: MovementType.DOWN,
        3: MovementType.UP,
        1: MovementType.LEFT,
        -1: MovementType.RIGHT
    }.get(diff, None)

# Función para imprimir la solución encontrada
def print_solution(goal_node, visited_set, root_node, expanded_nodes):
    path = goal_node.path()
    total_cost = 0
    print("\n======= SOLUCIÓN =======\n")
    for i in range(len(path)):
        current = path[i]
        print("*******")
        print("*", current.state[0:3], "*")
        print("*", current.state[3:6], "*")
        print("*", current.state[6:9], "*")
        print("*******")

        if i > 0:
            move_tile = path[i].state[path[i-1].state.index('0')]
            move_type = find_transition(path[i-1].state, path[i].state)
            cost = int(move_tile)
            total_cost += cost
            print(f"Movimiento: {move_tile} {move_type.name}")
            print(f"Costo del movimiento: {cost}\n")

    print("===== ESTADÍSTICAS =====")
    print("Transiciones:", len(path) - 1)
    print("Estados visitados:", len(visited_set))
    print("Costo total:", total_cost)
    print("Nodos expandidos:", expanded_nodes)
    print("========================\n")




# DFS (Búsqueda en profundidad)
def dfs(start_state, goal_state, max_depth=20):
    visited = set()
    stack = [Node(start_state)]
    expanded = 0

    while stack:
        node = stack.pop()
        expanded += 1

        if node.state == goal_state:
            print_solution(node, visited, stack[0], expanded)
            return

        if node.depth < max_depth:
            visited.add(node.state)
            for s in get_successors(node.state):
                if s not in visited:
                    child = Node(s, parent=node, cost=node.cost + 1, depth=node.depth + 1)
                    stack.append(child)

    print("No se encontró solución.")

# BFS (Búsqueda por amplitud)
def bfs(start_state, goal_state):
    visited = set()
    q = Queue()
    root = Node(start_state)
    q.put(root)
    expanded = 0

    while not q.empty():
        node = q.get()
        expanded += 1
        visited.add(node.state)

        if node.state == goal_state:
            print_solution(node, visited, root, expanded)
            return

        for s in get_successors(node.state):
            if s not in visited:
                child = Node(s, parent=node, cost=node.cost + 1, depth=node.depth + 1)
                q.put(child)

    print("No se encontró solución.")

# UCS (Búsqueda de costo uniforme)
def uniform_cost_search(start_state, goal_state):
    visited = set()
    pq = []
    root = Node(start_state)
    heapq.heappush(pq, (root.cost, root))
    expanded = 0

    while pq:
        _, node = heapq.heappop(pq)
        expanded += 1
        visited.add(node.state)

        if node.state == goal_state:
            print_solution(node, visited, root, expanded)
            return

        for s in get_successors(node.state):
            if s not in visited:
                child = Node(s, parent=node, cost=node.cost + 1, depth=node.depth + 1)
                heapq.heappush(pq, (child.cost, child))

    print("No se encontró solución.")

# Best First Search
def best_first_search(start_state, goal_state):
    def heuristic(state):
        return sum(1 for i in range(9) if state[i] != goal_state[i] and state[i] != '0')

    visited = set()
    pq = []
    root = Node(start_state)
    root.estimated_cost_to_goal = heuristic(start_state)
    heapq.heappush(pq, (root.estimated_cost_to_goal, root))
    expanded = 0

    while pq:
        _, node = heapq.heappop(pq)
        expanded += 1
        visited.add(node.state)

        if node.state == goal_state:
            print_solution(node, visited, root, expanded)
            return

        for s in get_successors(node.state):
            if s not in visited:
                child = Node(s, parent=node, cost=node.cost + 1, depth=node.depth + 1)
                child.estimated_cost_to_goal = heuristic(s)
                heapq.heappush(pq, (child.estimated_cost_to_goal, child))

    print("No se encontró solución.")

# A* Search
def a_star(start_state, goal_state):
    def manhattan_distance(state):
        total_distance = 0
        for i, value in enumerate(state):
            if value != '0':
                goal_index = goal_state.index(value)
                total_distance += abs(i // 3 - goal_index // 3) + abs(i % 3 - goal_index % 3)
        return total_distance

    visited = set()
    pq = []
    root = Node(start_state)
    root.set_total_cost(0, manhattan_distance(start_state))
    heapq.heappush(pq, root)
    expanded = 0

    while pq:
        node = heapq.heappop(pq)
        expanded += 1
        visited.add(node.state)

        if node.state == goal_state:
            print_solution(node, visited, root, expanded)
            return

        for s in get_successors(node.state):
            if s not in visited:
                child = Node(s, parent=node, cost=node.cost + 1, depth=node.depth + 1)
                child.set_total_cost(child.cost, manhattan_distance(s))
                heapq.heappush(pq, child)

    print("No se encontró solución.")

# Iterative Deepening
def iterative_deepening(start_state, goal_state, max_depth=20):
    for depth in range(max_depth):
        print(f"Profundidad {depth}:")
        dfs(start_state, goal_state, max_depth=depth)



# DFS (Búsqueda en profundidad)
def dfs(start_state, goal_state, max_depth=20):
    visited = set()
    stack = [Node(start_state)]
    expanded = 0

    while stack:
        node = stack.pop()
        expanded += 1

        if node.state == goal_state:
            print_solution(node, visited, stack[0], expanded)
            return

        if node.depth < max_depth:
            visited.add(node.state)
            for s in get_successors(node.state):
                if s not in visited:
                    child = Node(s, parent=node, cost=node.cost + 1, depth=node.depth + 1)
                    stack.append(child)

    print("No se encontró solución.")

# BFS (Búsqueda por amplitud)
def bfs(start_state, goal_state):
    visited = set()
    q = Queue()
    root = Node(start_state)
    q.put(root)
    expanded = 0

    while not q.empty():
        node = q.get()
        expanded += 1
        visited.add(node.state)

        if node.state == goal_state:
            print_solution(node, visited, root, expanded)
            return

        for s in get_successors(node.state):
            if s not in visited:
                child = Node(s, parent=node, cost=node.cost + 1, depth=node.depth + 1)
                q.put(child)

    print("No se encontró solución.")

# UCS (Búsqueda de costo uniforme)
def uniform_cost_search(start_state, goal_state):
    visited = set()
    pq = []
    root = Node(start_state)
    heapq.heappush(pq, (root.cost, root))
    expanded = 0

    while pq:
        _, node = heapq.heappop(pq)
        expanded += 1
        visited.add(node.state)

        if node.state == goal_state:
            print_solution(node, visited, root, expanded)
            return

        for s in get_successors(node.state):
            if s not in visited:
                child = Node(s, parent=node, cost=node.cost + 1, depth=node.depth + 1)
                heapq.heappush(pq, (child.cost, child))

    print("No se encontró solución.")

# Best First Search
def best_first_search(start_state, goal_state):
    def heuristic(state):
        return sum(1 for i in range(9) if state[i] != goal_state[i] and state[i] != '0')

    visited = set()
    pq = []
    root = Node(start_state)
    root.estimated_cost_to_goal = heuristic(start_state)
    heapq.heappush(pq, (root.estimated_cost_to_goal, root))
    expanded = 0

    while pq:
        _, node = heapq.heappop(pq)
        expanded += 1
        visited.add(node.state)

        if node.state == goal_state:
            print_solution(node, visited, root, expanded)
            return

        for s in get_successors(node.state):
            if s not in visited:
                child = Node(s, parent=node, cost=node.cost + 1, depth=node.depth + 1)
                child.estimated_cost_to_goal = heuristic(s)
                heapq.heappush(pq, (child.estimated_cost_to_goal, child))

    print("No se encontró solución.")

# A* Search
def a_star(start_state, goal_state):
    def manhattan_distance(state):
        total_distance = 0
        for i, value in enumerate(state):
            if value != '0':
                goal_index = goal_state.index(value)
                total_distance += abs(i // 3 - goal_index // 3) + abs(i % 3 - goal_index % 3)
        return total_distance

    visited = set()
    pq = []
    root = Node(start_state)
    root.set_total_cost(0, manhattan_distance(start_state))
    heapq.heappush(pq, root)
    expanded = 0

    while pq:
        node = heapq.heappop(pq)
        expanded += 1
        visited.add(node.state)

        if node.state == goal_state:
            print_solution(node, visited, root, expanded)
            return

        for s in get_successors(node.state):
            if s not in visited:
                child = Node(s, parent=node, cost=node.cost + 1, depth=node.depth + 1)
                child.set_total_cost(child.cost, manhattan_distance(s))
                heapq.heappush(pq, child)

    print("No se encontró solución.")

# Iterative Deepening
def iterative_deepening(start_state, goal_state, max_depth=20):
    for depth in range(max_depth):
        print(f"Profundidad {depth}:")
        dfs(start_state, goal_state, max_depth=depth)



# Menú para seleccionar el algoritmo a ejecutar
def select_algorithm(start_state, goal_state):
    print("\nSelecciona el algoritmo de búsqueda:")
    print("1. BFS (Búsqueda por amplitud)")
    print("2. DFS (Búsqueda en profundidad)")
    print("3. UCS (Búsqueda de costo uniforme)")
    print("4. Best First Search")
    print("5. A* Search")
    print("6. Iterative Deepening")

    choice = int(input("Ingresa el número de tu opción: "))

    if choice == 1:
        print("Ejecutando BFS...")
        bfs(start_state, goal_state)
    elif choice == 2:
        print("Ejecutando DFS...")
        dfs(start_state, goal_state)
    elif choice == 3:
        print("Ejecutando UCS...")
        uniform_cost_search(start_state, goal_state)
    elif choice == 4:
        print("Ejecutando Best First Search...")
        best_first_search(start_state, goal_state)
    elif choice == 5:
        print("Ejecutando A* Search...")
        a_star(start_state, goal_state)
    elif choice == 6:
        print("Ejecutando Iterative Deepening...")
        iterative_deepening(start_state, goal_state)
    else:
        print("Opción no válida.")

In [5]:
if __name__ == "__main__":
    # Estados de prueba
    EASY = "134862705"
    MEDIUM = "281043765"
    HARD = "567408321"
    GOAL = "123804765"

    # Menú de selección de algoritmo
    select_algorithm(MEDIUM, GOAL)



Selecciona el algoritmo de búsqueda:
1. BFS (Búsqueda por amplitud)
2. DFS (Búsqueda en profundidad)
3. UCS (Búsqueda de costo uniforme)
4. Best First Search
5. A* Search
6. Iterative Deepening
Ingresa el número de tu opción: 1
Ejecutando BFS...

======= SOLUCIÓN =======

*******
* 281 *
* 043 *
* 765 *
*******
*******
* 081 *
* 243 *
* 765 *
*******
Movimiento: 2 DOWN
Costo del movimiento: 2

*******
* 801 *
* 243 *
* 765 *
*******
Movimiento: 8 LEFT
Costo del movimiento: 8

*******
* 810 *
* 243 *
* 765 *
*******
Movimiento: 1 LEFT
Costo del movimiento: 1

*******
* 813 *
* 240 *
* 765 *
*******
Movimiento: 3 UP
Costo del movimiento: 3

*******
* 813 *
* 204 *
* 765 *
*******
Movimiento: 4 RIGHT
Costo del movimiento: 4

*******
* 813 *
* 024 *
* 765 *
*******
Movimiento: 2 RIGHT
Costo del movimiento: 2

*******
* 013 *
* 824 *
* 765 *
*******
Movimiento: 8 DOWN
Costo del movimiento: 8

*******
* 103 *
* 824 *
* 765 *
*******
Movimiento: 1 LEFT
Costo del movimiento: 1

*******
* 123 